In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import torchvision.transforms as transforms


In [3]:
#设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def corr2d(X, K):
    """计算二维互相关运算"""
    h,w=K.shape   
    Y=torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
    return Y


In [5]:
X=torch.tensor([[0,1,2],[3,4,5],[6,7,8]],dtype=torch.float32)
K=torch.tensor([[0,1],[2,3]],dtype=torch.float32)
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

In [6]:
#实现卷积层
class Conv2D(nn.Module):
    def _init(self,kernel_size):
        super().__init__()
        self.weight=nn.Parameter(torch.rand(kernel_size))
        self.bias=nn.Parameter(torch.zeros(1))
        
    def forward(self,X):
        return corr2d(X,self.weight)+self.bias

In [8]:
#实现利用卷积层完成图像边缘的检测
X=torch.ones((6,8),dtype=torch.float32)
X[:,2:6]=0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [10]:
K=torch.tensor([[1.0, -1.0]])
Y=corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

对于卷积核[1.0，-1.0]，如果是相同的连续数字，则输出为0，若是不想同的数字，则输出为1.0或-1.0
也就由此产生了图像边缘的检测

In [12]:
#学习由X生成Y的卷积核
conv2d=nn.Conv2d(in_channels=1,out_channels=1,kernel_size=(1,2),bias=False)
x=X.reshape((1,1,6,8))
y=Y.reshape((1,1,6,7))  

for i in range(100):
    y_hat=conv2d(x)
    l=(y_hat-y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:]-=3e-2*conv2d.weight.grad
    if (i+1) % 10 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}, weight {conv2d.weight.data.reshape(-1)}')
        
        
#输出卷积核
conv2d.weight.data.reshape(-1)

        

epoch 10, loss 0.305, weight tensor([ 0.9308, -1.0440])
epoch 20, loss 0.003, weight tensor([ 0.9938, -1.0059])
epoch 30, loss 0.000, weight tensor([ 0.9993, -1.0007])
epoch 40, loss 0.000, weight tensor([ 0.9999, -1.0001])
epoch 50, loss 0.000, weight tensor([ 1.0000, -1.0000])
epoch 60, loss 0.000, weight tensor([ 1.0000, -1.0000])
epoch 70, loss 0.000, weight tensor([ 1.0000, -1.0000])
epoch 80, loss 0.000, weight tensor([ 1.0000, -1.0000])
epoch 90, loss 0.000, weight tensor([ 1.0000, -1.0000])
epoch 100, loss 0.000, weight tensor([ 1.0000, -1.0000])


tensor([ 1.0000, -1.0000])